## GBDT+LR模型详解
> * 基于梯度提升算法的学习器叫做GBM(Gradient Boosting Machine)。理论上，GBM可以选择各种不同的学习算法作为基学习器。现实中，用得最多的基学习器是决策树。为什么梯度提升方法倾向于选择决策树（通常是CART树）作为基学习器呢？这与决策树算法自身的优点有很大的关系。决策树可以认为是if-then规则的集合，易于理解，可解释性强，预测速度快。同时，决策树算法相比于其他的算法需要更少的特征工程，比如可以不用做特征标准化，可以很好的处理字段缺失的数据，也可以不用关心特征间是否相互依赖等。决策树能够自动组合多个特征，它可以毫无压力地处理特征间的交互关系并且是非参数化的，因此你不必担心异常值或者数据是否线性可分（举个例子，决策树能轻松处理好类别A在某个特征维度x的末端，类别B在中间，然后类别A又出现在特征维度x前端的情况）。不过，单独使用决策树算法时，有容易过拟合缺点。所幸的是，通过各种方法，抑制决策树的复杂性，降低单颗决策树的拟合能力，再通过梯度提升的方法集成多个决策树，最终能够很好的解决过拟合的问题。

## 一、DT:Regression Decision Tree 回归决策树
> * 提起决策树也许大部分人，想到的都只是C4.5的分类决策树，但是还有回归决策树，而GBDT中的树全部是回归决策树。
> * 决策树分为两大类，回归树和分类树
> > 1. 回归树用于预测实数值，如明天的温度、用户的年龄、网页的相关程度等
> > 1. 分类树用于分类标签值，如晴天/雨天／雾天等，用户性别
> > 1. 回归树的结果相加减是有意义的，如10岁+5岁-3岁＝12岁。而分类树的结果相加减则没有意义，如男+男+女＝到底是男还是女

**GBDT的核心在于累加所有树的结果作为最终结果，就像前面对年龄的累加，而分类树的结果显然是没办法累加的，所以GBDT中的树都是回归树，不是分类树。**

那么回归树如何工作的呢？

* 下面我们以对人的性别判别／年龄预测为例来说明，每个instance都是一个我们已知的性别／年龄的人，而feature则包括这个人上网的时长、上网的时段、网购所花的金额等。

> * 作为对比，先说分类树，我们知道C4.5分类树在每次分枝时，是穷举每一个feature的每一个阀值，找到使得按照feature<=阀值，和feature>阀值分成的两个分枝的熵最大的feature和阀值。按照这个标准分枝得到两个新节点，用同样的方法继续分枝，直到所有人都被分入性别唯一的叶子节点，或者达到预设的终止条件。若最终的叶子节点中的性别不唯一，则以多数人的性别作为该叶子节点的性别。

> * 回归树的总体流程也类似，不过回归树在每一个节点（不一定非要是叶子节点）都会得到一个预测值。以年龄为例，该预测值等于属于这个节点的所有人的年龄的平均值。分枝时穷举每一个feature的每个阀值找最好的分割点，**但此时衡量最好的标准不再是最大熵，而是最小化均方差（也就是每个人的预测误差平方和除以N）**

GBDT调整之后可以用于分类问题，但是内部还是回归树。回归树用的是最小化均方误差，分类树用的是最小化基尼指数(CART)

## 二、GB:梯度迭代 Gradient Boosting
* 首先Boosting是一种集成方法。通过对弱分类器的组合得到强分类器，他是串行的，几个弱分类器之间是依次训练的。GBDT的核心就在于，每一颗树学习的是之前所有树结论和的残差。具体过程如下：
> * 主要是通过迭代多棵树来共同决策，实现过程为：比如A这个人，第一棵树认为是10岁，第二棵树认为是0岁，第三棵树认为是20岁，我们并不是取这三棵树的平均值10来作为最终评论，这种投票方法并不是GBDT。**GBDT是把所有树的结论累加起来做最终的结论，所以可以想到每棵树的结论并不是年龄本身，而是年龄的一个累加量。GBDT的核心就在于，每一棵树学的是之前所有树的结论和残差，这个残差就是一个加预测值后得到真实值的累加量。**比如A的真是年龄是18岁，但第一棵树的预测年龄是12岁，差了6岁，即残差为6岁，那么第二棵树里我们把A的年龄设为6岁去学习，如果第二棵树真的能把A分到6岁的叶子节点，那累加两棵树的结论就是A的真实年龄。如果第二棵树的结论是5岁，则A仍然存在1岁的残差，第三棵树里A的年龄就变成一岁，继续学。这就是Gradient Boosting在GBDT中的意义。
> * Gradient体现在：无论前面一棵树的cost function是什么，是均方误差还是均差，只要他以误差作为衡量标准，那么残差向量就是他的全局最优方向，这就是Gradient。

## 三、GBDT工作过程实例
还是年龄预测，简单起见训练集只有四个人，A,B,C,D.他们的年龄分别是14，16，24，26.其中A,B分别是高一和高三学生；C和D分别是应届毕业生和工作两年的员工。如果是用一棵传统的回归决策树来训练，会得到如下图1所示的结果：
![图一](http://img.blog.csdn.net/20130705172734000?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvdzI4OTcxMDIz/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

现在我们来使用GBDT来做这件事，由于数据太少，我们限定叶子节点最多有两个，即每棵树都只有一个分枝，并且限定只学两棵树。我们会得到如下图2所示结果：
![图二](http://img.blog.csdn.net/20130705172803234?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvdzI4OTcxMDIz/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

在第一棵树中的分枝和图一一样，由于A,B年龄较为相近，C,D年龄较为相近，他们被分为两波，每波用平均年龄作为预测值。此时计算残差(**残差的意思就是：A的预测值 ＋ A的残差 ＝ A的实际值**)，所以A的残差就是16-15=1。（**注意，这里A的预测值是指前面所有树累加的和，这里前面只有一棵树所以直接是15，如果还有树则需要都累加起来作为A的预测值**）.进而得出A,B,C,D的残差分别为－1，1，－1，1.然后我们那残差替代A,B,C,D的原值，到第二棵树去学习，如果我们的预测值和他们的残差相等，则只需要把第二棵树的结论累加到第一棵树上就能得到真实年龄了。这里的数据显然是我可以做的，第二棵树只有两个值1和－1，直接分为两个节点。此时所有人的残差都是0，即每个人都得到了真实的预测值。

那么哪里体现了Gradient呢？其实回到第一棵树结束时想一想，无论此时的cost function是什么，是均方误差还是均差，只要它以误差作为衡量标准，残差向量（－1，1，－1，1）都是他的全局最优方向。这就是Gradient.

**既然图一和图二的最终效果相同，为何还需要GBDT呢，答案是过拟合，我们发现图一为了达到100%精度使用了3个feature。相对来说图二的boosting虽然用了两棵树，但其实只用了2个feature就搞定了，显然图二的依据更靠谱。**

**Boosting的最大好处在于，每一步的残差计算其实变相地加大了分错instance的权重，而已经分对的instance则都趋向于0.这样后面的树就能越来越专注于那些前面被分错的instance。**

## 五、GBDT+LR

单独的使用GBDT模型，容易出现过拟合，在实际应用中往往使用 GBDT＋LR的方式做模型训练，算法更多细节可以参考 [Practical Lessons from Predicting Clicks on Ads at Facebook]。

首先根据样本训练出GBDT树，对于每个叶子节点，回溯到根节点都可以得到一组组合特征，所以用叶子节点的标号可以代表一个新的组合特征。结合上面的图，用一个样本为例，直观的表达如下：
<img width="650" height="650" src="http://git.intra.weibo.com/LiveAlgorithm/AlgorithmDocs/raw/master/GBDT.jpg"/>
其中 0号 组合特征的含义是：ageLessThan15AndIsMale，该样本取值 0
其中 1号 组合特征的含义是：ageLessThan15AndIsNotMale，该样本取值 1
其中 2号 组合特征的含义是：ageLargerOrEqualThan15，该样本取值 0
其中 3号 组合特征的含义是：useComputerDaily，该样本取值 0
其中 4号 组合特征的含义是：notUseComputerDaily，该样本取值 1

这部分特征是GBDT生成的组合特征，再结合LR固有的稀疏特征，就组成了 GBDT ＋ LR 模型。生成样本向量阶段，样本首先过GBDT模型，生成组合特征部分的输入向量，再结合固有的稀疏特征向量，组成新的特征向量，示例如下：
<img width="650" height="650" src="http://git.intra.weibo.com/LiveAlgorithm/AlgorithmDocs/raw/master/GBDTLR.jpg"/>
在该例子中，第一行绿颜色是通过 GBDT 模型生成的特征向量，每个值都代表一个叶子节点的输出（样本在某棵树只在一个叶子节点有输出），第二行表示 LR 模型的稀疏特征向量，第三行表示把两部分特征向量拼接在一起，组成一个最终的特征向量，并使用该向量训练LR模型。


